In [ ]:
import pandas as pd
from tweepy.client import Client
import tweepy
import time

bearer_token = "FILL IN"
API_Key = "FILL IN"
API_secret_key = "FILL IN"
client = Client(bearer_token = bearer_token, 
                consumer_key = API_Key,
                consumer_secret = API_secret_key,
                wait_on_rate_limit = False,
               return_type = dict)

# merge old and new data and save as csv
def merge_data(new_data, old_data, filename):
    merged_data = pd.concat([old_data, new_data])
    merged_data.to_csv(filename, sep = "|")
    return merged_data

**GET USER FOLLOWERS UKRAINE DATA**

In [ ]:
# start time
start_time = time.time()

# initiate filename
filename = "users_April_Ukraine.csv"

# read data list of ids
user_list = pd.read_csv("userID_list.csv").drop(columns = ['Unnamed: 0'])
user_list['id']

# initiate start + end id
start_id = 0
end_id = 100

# index of last user id 
last_id = len(user_list) - 1

# set finished on False
finished = False

# get first users
ids = [item for item in user_list[start_id:end_id].id]
old_users = client.get_users(ids = ids, user_fields = ["created_at", "protected", "public_metrics", "verified"])
old_users_csv = pd.json_normalize(old_users['data'])
old_users_csv['username'] = old_users_csv.apply(lambda row: row['username'].replace("|", ""), axis = 1) # remove | from username
old_users_csv['name'] = old_users_csv.apply(lambda row: row['name'].replace("|", ""), axis = 1) # remove | from name

# start user lookup
while finished == False:
    
    # update and inform about ids
    print("number of ids: ", end_id)
    start_id = start_id + 100
    end_id = end_id + 100
    
    # make sure to get to the final id
    if end_id > last_id:
        end_id = last_id 
    
    # make sure to stop when done
    if start_id >= last_id:
        finished = True
    
    # get 100 ids
    ids = [item for item in user_list[start_id:end_id].id]
    
    # get user data
    new_users = client.get_users(ids = ids, user_fields = ["created_at", "protected", "public_metrics", "verified"])
    
    # update csv
    new_users_csv = pd.json_normalize(new_users['data'])
    new_users_csv['username'] = new_users_csv.apply(lambda row: row['username'].replace("|", ""), axis = 1) # remove | from username
    new_users_csv['name'] = new_users_csv.apply(lambda row: row['name'].replace("|", ""), axis = 1) # remove | from name
    
    old_users_csv = merge_data(new_users_csv, old_users_csv, filename)
    
    # wait 3 seconds
    time.sleep(3)
    
# end timer
end_time = time.time()
print("time elapsed in minutes: ", (end_time - start_time)/60)   


**GET COVID DATA**

In [ ]:
# start timer
start_timer = time.time()

# initiate start and end times
start_before_invasion = '2022-02-21T02:30:00Z'
invasion_time = '2022-02-24T02:30:00Z'
right_after_invasion_time = '2022-02-24T03:18:00Z'
end_after_invasion = '2022-02-25T02:30:00Z'

for item in ['before', 'after']:
    
    if item == 'before':
        print("Scraping tweets from before the invasion")
        
        # initiate start and end time
        start_time = start_before_invasion
        end_time = invasion_time
        
        # initiate filenames
        filename_tweets = 'covid_before_tweets.csv'
        filename_users = 'covid_before_users.csv'
        
    if item == 'after':
        print("Scraping tweets from after the invasion")
        
        # initiate start and end time
        start_time = right_after_invasion_time
        end_time = end_after_invasion
        
        # initiate filenames
        filename_tweets = 'covid_after_tweets.csv'
        filename_users = 'covid_after_users.csv'
        
    tweets = client.search_all_tweets(query = '(covid OR #covid OR #Covid OR #COVID) lang:en -is:reply -is:quote -is:retweet', expansions = ["author_id"], 
                           user_fields = ["created_at", "public_metrics", "verified"], 
                           tweet_fields = ["author_id", "created_at", "public_metrics", "lang", "entities", "conversation_id", "referenced_tweets"],
                        start_time = start_time, end_time = end_time, max_results = 500)

    # store dataframes
    old_tweets = pd.json_normalize(tweets['data'])
    old_tweets['text'] = old_tweets.apply(lambda row: row['text'].replace("|", ""), axis = 1) # remove | from tweets

    old_users = pd.json_normalize(tweets['includes']['users'])
    old_users['username'] = old_users.apply(lambda row: row['username'].replace("|", ""), axis = 1) # remove | from username
    old_users['name'] = old_users.apply(lambda row: row['name'].replace("|", ""), axis = 1) # remove | from name

    # go through 
    while ('meta' in tweets):
        
        # break if there is no next token
        if 'next_token' not in tweets['meta']:
            break
            
        # print update
        print("total number of tweets: ", len(old_tweets))

        # sleep for 3 seconds to prevent reaching the rate limit
        time.sleep(3)
        
        # get next token
        next_token = tweets['meta']['next_token']
        tweets = client.search_all_tweets(query = '(covid OR #covid OR #Covid OR #COVID) lang:en -is:reply -is:quote -is:retweet', expansions = ["author_id"], 
                               user_fields = ["created_at", "public_metrics", "verified"], 
                               tweet_fields = ["author_id", "created_at", "public_metrics", "lang", "entities", "conversation_id", "referenced_tweets"],
                            start_time = start_time, end_time = end_time, next_token = next_token, max_results = 500)

        # update tweet csv
        new_tweets = pd.json_normalize(tweets['data'])
        new_tweets['text'] = new_tweets.apply(lambda row: row['text'].replace("|", ""), axis = 1) # remove | from tweets
        old_tweets = merge_data(new_tweets, old_tweets, filename_tweets)

        # update users csv
        new_users = pd.json_normalize(tweets['includes']['users'])
        new_users['username'] = new_users.apply(lambda row: row['username'].replace("|", ""), axis = 1) # remove | from username
        new_users['name'] = new_users.apply(lambda row: row['name'].replace("|", ""), axis = 1) # remove | from name
        old_users = merge_data(new_users, old_users, filename_users)
        

# end timer
end_time = time.time()
print("time in sec elapsed: ", end_time - start_timer)
print("time in min elapsed: ", (end_time - start_timer) / 60)

**EXTRA COVID DATA**

In [ ]:
# start timer
start_timer = time.time()

# initiate start and end times
start_time = '2022-02-25T02:30:00Z'
end_time = '2022-02-27T02:30:00Z'

filename_tweets = 'covid_afterEXTRA_tweets.csv'
filename_users = 'covid_afterEXTRA_users.csv'

tweets = client.search_all_tweets(query = '(covid OR #covid OR #Covid OR #COVID) lang:en -is:reply -is:quote -is:retweet', expansions = ["author_id"], 
                           user_fields = ["created_at", "public_metrics", "verified"], 
                           tweet_fields = ["author_id", "created_at", "public_metrics", "lang", "entities", "conversation_id", "referenced_tweets"],
                        start_time = start_time, end_time = end_time, max_results = 500)

# store dataframes
old_tweets = pd.json_normalize(tweets['data'])
old_tweets['text'] = old_tweets.apply(lambda row: row['text'].replace("|", ""), axis = 1) # remove | from tweets

old_users = pd.json_normalize(tweets['includes']['users'])
old_users['username'] = old_users.apply(lambda row: row['username'].replace("|", ""), axis = 1) # remove | from username
old_users['name'] = old_users.apply(lambda row: row['name'].replace("|", ""), axis = 1) # remove | from name

# go through 
while ('meta' in tweets):

    # break if there is no next token
    if 'next_token' not in tweets['meta']:
        break

    # print update
    print("total number of tweets: ", len(old_tweets))

    # sleep for 3 seconds to prevent reaching the rate limit
    time.sleep(3)

    # get next token
    next_token = tweets['meta']['next_token']
    tweets = client.search_all_tweets(query = '(covid OR #covid OR #Covid OR #COVID) lang:en -is:reply -is:quote -is:retweet', expansions = ["author_id"], 
                           user_fields = ["created_at", "public_metrics", "verified"], 
                           tweet_fields = ["author_id", "created_at", "public_metrics", "lang", "entities", "conversation_id", "referenced_tweets"],
                        start_time = start_time, end_time = end_time, next_token = next_token, max_results = 500)

    # update tweet csv
    new_tweets = pd.json_normalize(tweets['data'])
    new_tweets['text'] = new_tweets.apply(lambda row: row['text'].replace("|", ""), axis = 1) # remove | from tweets
    old_tweets = merge_data(new_tweets, old_tweets, filename_tweets)

    # update users csv
    new_users = pd.json_normalize(tweets['includes']['users'])
    new_users['username'] = new_users.apply(lambda row: row['username'].replace("|", ""), axis = 1) # remove | from username
    new_users['name'] = new_users.apply(lambda row: row['name'].replace("|", ""), axis = 1) # remove | from name
    old_users = merge_data(new_users, old_users, filename_users)


# end timer
end_time = time.time()
print("time in sec elapsed: ", end_time - start_timer)
print("time in min elapsed: ", (end_time - start_timer) / 60)